In [1]:
import torch
from torchsummary import summary
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from pathlib import Path
import wandb

import sys

sys.path.append("../src")

from trainer import Trainer
from module import VGGNetModule
from utils import model_size
from callbacks import OverfitCallback, EarlyStoppingCallback
from logger import WandbLogger

In [2]:
data_path = Path("../data")
logs_path = Path("../logs")
logs_path.mkdir(exist_ok=True)

In [3]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "ConvNet",
        "batch_size": 64,
        "max_epochs": 50,
        "optimizer": {
            "name": "Adam",
        },
        "train_split": 42_000,
        "val_split": 8000
    },
    logs_path=logs_path
)

In [4]:
dataset = CIFAR10(data_path, train=True, download=True, transform=v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
]))

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]])

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=logger.config["batch_size"])

Files already downloaded and verified


In [5]:
callbacks = [
    EarlyStoppingCallback(min_val_accuracy=60.0),
    OverfitCallback(limit_batches=2, max_epochs=200),
]

In [ ]:
module = VGGNetModule()

trainer = Trainer(
    module=module,
    logger=logger,
    callbacks=callbacks,
    logs_path=logs_path,
    fast_dev_run=False,
    measure_time=True
)

model_size(module.model)

model size: 0.02 MB


In [7]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


Time per epoch: 26.76 seconds
Epoch: 0, train_accuracy: 33.35, val_accuracy: 43.42
Epoch: 1, train_accuracy: 47.32, val_accuracy: 48.53
Epoch: 2, train_accuracy: 51.61, val_accuracy: 50.92
Epoch: 3, train_accuracy: 54.72, val_accuracy: 55.50
Epoch: 4, train_accuracy: 56.75, val_accuracy: 55.85
Epoch: 5, train_accuracy: 58.42, val_accuracy: 57.67
Epoch: 6, train_accuracy: 59.57, val_accuracy: 59.00
Epoch: 7, train_accuracy: 60.72, val_accuracy: 59.44
Epoch: 8, train_accuracy: 61.48, val_accuracy: 60.69
Early stopping check started!
Epoch: 9, train_accuracy: 61.90, val_accuracy: 60.84
Epoch: 10, train_accuracy: 62.84, val_accuracy: 60.86
Epoch: 11, train_accuracy: 63.45, val_accuracy: 61.67
Epoch: 12, train_accuracy: 63.82, val_accuracy: 62.01
Epoch: 13, train_accuracy: 64.28, val_accuracy: 62.50
Epoch: 14, train_accuracy: 64.70, val_accuracy: 62.95
Epoch: 15, train_accuracy: 65.40, val_accuracy: 63.29
Epoch: 16, train_accuracy: 65.39, val_accuracy: 62.49
Epoch: 17, train_accuracy: 65.66

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
epoch_train_accuracy,▁▄▅▆▆▆▇▇▇▇▇███████
epoch_train_loss,█▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
epoch_val_accuracy,▁▃▄▅▅▆▆▇▇▇▇▇██████
epoch_val_loss,█▆▅▄▄▃▂▃▂▂▂▂▁▁▁▁▁▁
step,▂▁▃▄▆▆▆▆▁▃▇▃▃▅▇▄▂▃▅▅▁▁▄▆█▆▇▅█▃▄▇▇▁▂▄▆▁▆█
epoch,17
epoch_train_accuracy,65.66306
epoch_train_loss,0.95328
epoch_val_accuracy,62.225
epoch_val_loss,1.05013


[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)